# **Geometric Graph Neural Networks (GNNs) Walkthrough**


**Objectives:**
1. Understand the theoretical foundations of geometric graphs and GNNs.
2. Implement a custom Geometric GNN step-by-step.
3. Explore advanced geometric concepts such as equivariance and positional embeddings.
4. Train and test a GNN on a geometric dataset.

---

## **Section 1: Introduction**
Geometric Graph Neural Networks extend traditional GNNs by incorporating geometric information such as node positions, distances, or rotational symmetries.

### **Applications of Geometric GNNs**
- **Molecular modeling:** Predicting molecular properties by modeling atomic interactions.
- **3D Computer Vision:** Analyzing point clouds or meshes for object recognition.
- **Robotics:** Path planning with geometric constraints.

---

In [ ]:
!pip install torch_geometric torch torchvision torch-cluster -q
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch_geometric.nn import knn_graph
import torch_cluster


In [ ]:
pip show torch-cluster

Name: torch_cluster
Version: 1.6.3+pt25cu121
Summary: PyTorch Extension Library of Optimized Graph Cluster Algorithms
Home-page: https://github.com/rusty1s/pytorch_cluster
Author: Matthias Fey
Author-email: matthias.fey@tu-dortmund.de
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: scipy
Required-by: 


---

## **Section 2: Mathematical Foundations**
Here, we will explore the theoretical foundations of graphs, geometric information, and their integration into GNNs.

### **Key Definitions**
- **Graph**: A graph \( G = (V, E) \) consists of:
  - \( V \): A set of nodes (vertices).
  - \( E \): A set of edges, which may be weighted or directed.
- **Graph Laplacian**:
  \[
  \mathbf{L} = \mathbf{D} - \mathbf{A}
  \]
  Where \( \mathbf{D} \) is the degree matrix and \( \mathbf{A} \) is the adjacency matrix.

### **Geometry in Graphs**
Graphs embedded in geometric spaces (e.g., 3D point clouds) have additional information:
- **Node coordinates**: \( x_v \in \mathbb{R}^d \) (e.g., 3D positions).
- **Edge distances**: Euclidean distances \( ||x_i - x_j||_2 \).
- **Intrinsic/Extrinsic Geometry**:
  - **Intrinsic**: Properties dependent only on graph topology.
  - **Extrinsic**: Properties involving embedding in a space (e.g., Euclidean distance).

Let us construct a simple geometric graph to visualize these concepts.
---



In [ ]:
# Section 2: Creating a geometric graph
import numpy as np

# Generate random 3D coordinates for 100 nodes
num_nodes = 100
coordinates = np.random.rand(num_nodes, 3)

# Use PyTorch Geometric to compute k-NN graph
edge_index = torch_cluster.knn_graph(torch.tensor(coordinates, dtype=torch.float), k=5)

# Define node features and labels
x = torch.tensor(coordinates, dtype=torch.float)  # Node features: coordinates
y = torch.randint(0, 2, (num_nodes,))  # Random binary labels

# Create a PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index, y=y)

print("Graph data:")
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")


Graph data:
Number of nodes: 100
Number of edges: 500


---

## **Section 3: Message Passing and Geometric GNNs**
Traditional GNNs rely on the **Message Passing Framework**, which aggregates information from a node’s neighbors:

$$
\mathbf{h}_v^{(k+1)} = \text{AGGREGATE}(\{\mathbf{h}_u^{(k)}: u \in \mathcal{N}(v)\})
$$

### **Limitations of Standard GNNs**
- Cannot exploit spatial relationships (e.g., node coordinates).
- Sensitive to rotation or translation of geometric graphs.
- Struggles with high-dimensional node features like 3D positions.

To address these, we introduce **Geometric GNNs**:
1. Incorporate geometric features (e.g., distances).
2. Use equivariant or invariant layers to handle transformations.

---


In [ ]:
# Section 3: Defining Geometric Features
def compute_edge_features(data):
    # Compute pairwise Euclidean distances
    row, col = data.edge_index
    edge_attr = (data.x[row] - data.x[col]).norm(dim=1, keepdim=True)
    return edge_attr

data.edge_attr = compute_edge_features(data)
print(f"Edge features (distances) added. Shape: {data.edge_attr.shape}")


Edge features (distances) added. Shape: torch.Size([500, 1])


### **Custom Geometric GNN Layer**
The layer will:
1. Aggregate node and edge features.
2. Use pairwise distances as edge attributes.
3. Pass combined features through an MLP.

---


In [ ]:
# Section 3: Custom Geometric GNN Layer
class GeometricGNN(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GeometricGNN, self).__init__(aggr='mean')  # Mean aggregation
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(in_channels * 2 + 1, 64),  # Include edge distances
            torch.nn.ReLU(),
            torch.nn.Linear(64, out_channels)
        )

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_i, x_j, edge_attr):
        # Concatenate central node, neighbor, and edge features
        interaction = torch.cat([x_i, x_j, edge_attr], dim=-1)
        return self.mlp(interaction)

# Test layer
layer = GeometricGNN(in_channels=3, out_channels=16)
out = layer(data.x, data.edge_index, data.edge_attr)
print(f"Output of GeometricGNN layer: {out.shape}")


Output of GeometricGNN layer: torch.Size([100, 16])


---

## **Section 4: Training a Geometric GNN**
Now that we’ve defined our geometric GNN layer, let’s build a model and train it on the synthetic dataset.

### **Model Architecture**
1. Two geometric GNN layers.
2. Final classifier for binary classification.

---


In [ ]:
# Section 4: Model Definition
class GeometricGraphNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GeometricGraphNet, self).__init__()
        self.gnn1 = GeometricGNN(input_dim, hidden_dim)
        self.gnn2 = GeometricGNN(hidden_dim, hidden_dim)
        self.classifier = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.gnn1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.gnn2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

model = GeometricGraphNet(input_dim=3, hidden_dim=64, output_dim=2)
print("Model architecture defined.")


Model architecture defined.


### **Training and Testing**
We will train the model using a standard cross-entropy loss and evaluate its accuracy.

---


In [ ]:
# Section 4: Training Loop
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = (pred == data.y).sum().item() / data.y.size(0)
    return acc

# Training loop
for epoch in range(50):
    loss = train()
    acc = test()
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Accuracy: {acc:.4f}")


Epoch 1, Loss: 0.6932, Accuracy: 0.5100
Epoch 2, Loss: 0.6927, Accuracy: 0.5400
Epoch 3, Loss: 0.6906, Accuracy: 0.5800
Epoch 4, Loss: 0.6870, Accuracy: 0.5800
Epoch 5, Loss: 0.6835, Accuracy: 0.5200
Epoch 6, Loss: 0.6864, Accuracy: 0.5500
Epoch 7, Loss: 0.6792, Accuracy: 0.5400
Epoch 8, Loss: 0.6865, Accuracy: 0.5400
Epoch 9, Loss: 0.6775, Accuracy: 0.5600
Epoch 10, Loss: 0.6824, Accuracy: 0.5500
Epoch 11, Loss: 0.6764, Accuracy: 0.5600
Epoch 12, Loss: 0.6748, Accuracy: 0.5600
Epoch 13, Loss: 0.6757, Accuracy: 0.5600
Epoch 14, Loss: 0.6706, Accuracy: 0.5300
Epoch 15, Loss: 0.6673, Accuracy: 0.5500
Epoch 16, Loss: 0.6647, Accuracy: 0.5600
Epoch 17, Loss: 0.6582, Accuracy: 0.5800
Epoch 18, Loss: 0.6561, Accuracy: 0.5800
Epoch 19, Loss: 0.6530, Accuracy: 0.5800
Epoch 20, Loss: 0.6541, Accuracy: 0.6000
Epoch 21, Loss: 0.6508, Accuracy: 0.6100
Epoch 22, Loss: 0.6489, Accuracy: 0.6200
Epoch 23, Loss: 0.6476, Accuracy: 0.6300
Epoch 24, Loss: 0.6450, Accuracy: 0.6200
Epoch 25, Loss: 0.6423, A

---

## **Section 5: Advanced Topics**
### **Dynamic Graphs**
- In tasks like 3D point clouds, edges may change dynamically.
- Example: Recomputing edges during training.

### **Equivariance**
- Geometric GNNs can be made equivariant to transformations.
- Example: \( E(3) \)-equivariant layers for molecules.

### **Transformers on Graphs**
- Positional embeddings help adapt transformers to geometric data.

**Future Directions**:
1. Scaling to large graphs.
2. Exploring advanced architectures like `e3nn`.

---

